In [70]:
import plaidml.keras
plaidml.keras.install_backend()

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.stattools import adfuller

from fbprophet import Prophet

import math

# import pyflux as pf

import warnings
warnings.filterwarnings(action='once')

import itertools

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import TimeSeriesSplit

from pandas.plotting import autocorrelation_plot

import re

import sys
import os

from functools import reduce

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM
#from keras.layers import Concatbenate
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot, plot_model
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop
from keras.models import load_model

import datetime as dt

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pickle

from numpy.random import seed

from tensorflow import set_random_seed

In [71]:
plt.rcParams['figure.dpi'] = 300

In [72]:
seed(2019)
set_random_seed(2019)

In [73]:
## Importing Data

In [74]:
df_combined = pd.read_pickle('./processed_data/df_combined.pickle')

df_combined.tail(2)

,ds,y,vix,gold
12666,2019-06-01 22:00:00,8568.81,30.447,3561.58
12667,2019-06-01 23:00:00,8560.63,30.447,3569.34


In [75]:
df_daily = df_combined[df_combined.ds.dt.hour == 0]

In [76]:
df_daily.tail(3)

,ds,y,vix,gold
12596,2019-04-01,4093.05,29.117,3648.05
12620,2019-05-01,5300.21,25.550,3729.78
12644,2019-06-01,8530.27,30.447,3567.73


In [77]:
df_daily.reset_index(drop = True, inplace = True)

In [78]:
df_daily.tail(2)

,ds,y,vix,gold
526,2019-05-01,5300.21,25.550,3729.78
527,2019-06-01,8530.27,30.447,3567.73


In [79]:
df_daily.tail()

,ds,y,vix,gold
523,2019-02-01,3413.16,38.667,3643.99
524,2019-03-01,3797.64,30.708,3685.30
525,2019-04-01,4093.05,29.117,3648.05
526,2019-05-01,5300.21,25.550,3729.78
527,2019-06-01,8530.27,30.447,3567.73


In [80]:
df_sentiment = pd.read_pickle('../sentiment_analysis/data_frames/mean_sent_subj_across_days_2_5_years_without_num.pickle')

In [81]:
df_sentiment.reset_index(inplace = True)

In [82]:
df_sentiment.head(2)

,date,article_polarity_neg_pos,article_subjectivity
0,2017-01-01,0.108259,0.398895
1,2017-01-02,0.155181,0.475218


In [83]:
df_sentiment.rename(columns = {'date': 'ds'}, inplace = True)

In [84]:
df_sentiment.ds = pd.to_datetime(df_sentiment.ds)

In [85]:
df_sentiment.head(2)

,ds,article_polarity_neg_pos,article_subjectivity
0,2017-01-01,0.108259,0.398895
1,2017-01-02,0.155181,0.475218


In [86]:
df_total = pd.merge(df_daily, df_sentiment, on='ds', how='outer')

In [87]:
df_total = df_total[0:528]

In [88]:
df_total = df_total.fillna(0)

In [89]:
df_total.drop('article_subjectivity', axis = 1, inplace = True)

In [90]:
df_total.head(3)

,ds,y,vix,gold,article_polarity_neg_pos
0,2017-02-08,1054.03,18.879,2732.00,0.041166
1,2017-02-09,1059.04,18.619,2703.51,0.046156
2,2017-02-10,986.42,18.165,2793.34,0.056675


In [91]:
def split_train_test_chronological(df, ratio = 0.9, use_ratio = True, index = 1000):
    
    '''
    Input is a dataframe, and a ratio. Splits dataframe into 2 dataframes chronologically.
    Returns first dataframe up to the index of the length of the input dataframe times the input ratio, 
    and returns second dataframe of remaining elements.
    use_ratio is a flag, wether ratio should be used or indicies instead.
    
    df = input dataframe
    ratio = ratio to be used for splitting
    use_ratio = if True, use ratio, 
    index = index to split input dataframe on
    
    '''
    if use_ratio:
        size = len(df) * ratio
        size_round = round(size)

        df_train = df[0:(size_round)]
        df_test = df[size_round:]
    else:
        df_train = df[0:(index)]
        df_test = df[index:]
    return df_train, df_test

In [92]:
def make_variables(df, lags, look_back, predict_window):
    
    '''
    In order to use keras LSTM, we need to convert the input into a keras-friendly input.
    
    df = input dataframe
    lags = number of lags
    look_back = number of preceding elements to be considered
    predict_window = size of window for predictions
    
    '''
    
    features = lags + 1
    start = look_back
    stop = len(df) - lags - predict_window

    lstm_in_X = np.zeros(shape=(stop-start, look_back+1, features))
    lstm_in_Y = np.zeros(shape=(stop-start, look_back+1))

    iter_list = [num for num in range(look_back+1)][::-1]
    for i in range(start, stop):
        for index, j in enumerate(iter_list):
            X = df[i - j : i - j + lags + 1, -1]
            lstm_in_X[i - start, index] = np.ravel(X)
            Y = df[i - j + lags + 1, -1]
            lstm_in_Y[i-start, index] = Y
            
    return lstm_in_X, lstm_in_Y, features

In [93]:
def train_lstm(train_X, train_Y, lags, look_back, predict_window, lstm_nodes,
               dense_layers, dropout = 0.1, loss_type = 'hinge', optimizer_type = 'adam',
               number_epochs = 300, batch_size = 24, ):
    
    
    model = Sequential()
    model.add(LSTM(lstm_nodes, input_shape=(look_back+1, features)))
    model.add(Dropout(dropout))
    for nodes in dense_layers:
        model.add(Dense(nodes))
        model.add(Dropout(dropout))
    model.add(Dense(look_back + 1))
    model.compile(loss=loss_type, optimizer= optimizer_type)
    model.fit(train_X, train_Y, epochs=number_epochs, batch_size = batch_size, verbose = 1)
    
    pred_Y_train = model.predict(train_X)
    predictions = pred_Y_train[:,-1]
    actuals = train_Y[:,-1]
    
    print(model.summary())
    
    return model, dataset, train_X, train_Y, predictions, actuals

In [94]:
def make_rolling_window(model, train_x, train_y, test_x, test_y, batch_size = 218, epochs = 2):
    predictions_test = []
    actuals_test = []
    yhats = []
    ys = []
    
    dimension = train_x.shape[2]
    
    for i in range(0, len(test_y) - 1):
        model.fit(train_x, train_y, 
                    epochs= epochs, 
                    batch_size = batch_size, 
                    validation_data=(test_x[i].reshape(1,1,dimension), (test_y[i].reshape(1 ,))),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)
        pred_Y_test = model.predict(test_x[i].reshape(1,1,dimension))
        train_x = np.concatenate((train_x, (test_x[i].reshape(1 , 1 , dimension))))

        train_y = np.concatenate((train_y, (test_y[i].reshape(1 ,))))
        predict_test = pred_Y_test[-1,-1]
        actual_test = train_y[-1]
        predictions_test.append(predict_test)
        actuals_test.append(actual_test)
        yhats.append(pred_Y_test)
        ys.append(test_y[i])
        
    return predictions_test, actuals_test, yhats, ys

In [95]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    '''
    
    
    '''
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [129]:
# load dataset
values = df_daily.drop('ds', axis = 1).values
# values = df_combined.drop('ds', axis = 1).values
# integer encode direction
# ensure all data is float
values = values.astype('float32')
# normalize features
# scaler = MinMaxScaler()
scaler = StandardScaler()
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 7, 1)
reframed.head()
# reframed_2 = series_to_supervised(values, 7, 1)
# reframed_2.head()

# drop columns we don't want to predict
y = reframed.iloc[:,-1].values
X = reframed.drop('var1(t)', axis =1).values
# split into train and test sets

n_train_days = 500
n_test_days = 10
train_X = X[:n_train_days,:]
train_y = y[:n_train_days]

val_X= X[n_train_days:-n_test_days,]
val_y= y[n_train_days:-n_test_days]


test_X = X[-n_test_days:,:]
test_y = y[-n_test_days:]



# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

train_and_val_X = np.concatenate((train_X, val_X), axis = 0)

train_and_val_y = np.concatenate((train_y, val_y), axis = 0)

print(train_X.shape, train_y.shape,val_X.shape, val_y.shape ,test_X.shape, test_y.shape)

(500, 1, 23) (500,) (11, 1, 23) (11,) (10, 1, 23) (10,)


In [130]:
LSTM_model_daily_1 = Sequential()
LSTM_model_daily_1.add(LSTM(64, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), dropout=0.20,recurrent_dropout=0.20))
# LSTM_model_daily_1.add(LSTM(128))
LSTM_model_daily_1.add(Dense(32))
LSTM_model_daily_1.add(Dense(32))
# LSTM_model_daily_1.add(LSTM(16, activation='relu'))
LSTM_model_daily_1.add(Dense(1))
LSTM_model_daily_1.compile(optimizer='adam', loss='mse')


In [131]:
history_daily_1 = LSTM_model_daily_1.fit(train_X, train_y, 
                    epochs= 100, 
                    batch_size = 32, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

Train on 500 samples, validate on 11 samples
Epoch 1/100
 - 0s - loss: 1.0209 - val_loss: 0.9742
Epoch 2/100
 - 0s - loss: 0.5145 - val_loss: 0.6215
Epoch 3/100
 - 0s - loss: 0.1744 - val_loss: 0.1829
Epoch 4/100
 - 0s - loss: 0.1928 - val_loss: 0.0686
Epoch 5/100
 - 0s - loss: 0.1367 - val_loss: 0.0085
Epoch 6/100
 - 0s - loss: 0.0980 - val_loss: 0.0069
Epoch 7/100
 - 0s - loss: 0.0986 - val_loss: 0.0050
Epoch 8/100
 - 0s - loss: 0.1082 - val_loss: 0.0093
Epoch 9/100
 - 0s - loss: 0.0930 - val_loss: 0.0074
Epoch 10/100
 - 0s - loss: 0.0778 - val_loss: 0.0022
Epoch 11/100
 - 0s - loss: 0.0881 - val_loss: 0.0027
Epoch 12/100
 - 0s - loss: 0.0909 - val_loss: 0.0101
Epoch 13/100
 - 0s - loss: 0.0734 - val_loss: 0.0026
Epoch 14/100
 - 0s - loss: 0.0819 - val_loss: 0.0082
Epoch 15/100
 - 0s - loss: 0.0774 - val_loss: 0.0050
Epoch 16/100
 - 0s - loss: 0.0703 - val_loss: 0.0126
Epoch 17/100
 - 0s - loss: 0.0586 - val_loss: 0.0039
Epoch 18/100
 - 0s - loss: 0.0654 - val_loss: 0.0019
Epoch 19/1

In [132]:
res_1 = make_rolling_window(LSTM_model_daily_1, train_X, train_y, val_X, val_y, batch_size = 32, epochs = 2)

Train on 500 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0413 - val_loss: 1.1078e-04
Epoch 2/2
 - 0s - loss: 0.0521 - val_loss: 0.0552
Train on 501 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0366 - val_loss: 8.3446e-05
Epoch 2/2
 - 0s - loss: 0.0338 - val_loss: 0.0283
Train on 502 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0343 - val_loss: 0.0012
Epoch 2/2
 - 0s - loss: 0.0412 - val_loss: 0.0398
Train on 503 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0322 - val_loss: 0.0012
Epoch 2/2
 - 0s - loss: 0.0407 - val_loss: 0.0154
Train on 504 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0352 - val_loss: 6.4111e-08
Epoch 2/2
 - 0s - loss: 0.0319 - val_loss: 0.0223
Train on 505 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0340 - val_loss: 0.0014
Epoch 2/2
 - 0s - loss: 0.0453 - val_loss: 0.0066
Train on 506 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0454 - val_loss: 0.0306
Epoch 2/2
 - 0s - loss: 0.0713

In [133]:
res_test = make_rolling_window(LSTM_model_daily_1, train_and_val_X, train_and_val_y, test_X, test_y, batch_size = 32, epochs = 2)



Train on 511 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0537 - val_loss: 0.0694
Epoch 2/2
 - 0s - loss: 0.0642 - val_loss: 0.0010
Train on 512 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0444 - val_loss: 0.0757
Epoch 2/2
 - 0s - loss: 0.0597 - val_loss: 1.5601e-04
Train on 513 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0583 - val_loss: 0.0825
Epoch 2/2
 - 0s - loss: 0.0696 - val_loss: 1.2528e-05
Train on 514 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0702 - val_loss: 0.0890
Epoch 2/2
 - 0s - loss: 0.1309 - val_loss: 0.0055
Train on 515 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0791 - val_loss: 0.2638
Epoch 2/2
 - 0s - loss: 0.0718 - val_loss: 0.0186
Train on 516 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0699 - val_loss: 0.2185
Epoch 2/2
 - 0s - loss: 0.0550 - val_loss: 0.0054
Train on 517 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0445 - val_loss: 0.0753
Epoch 2/2
 - 0s - loss: 0.0431 - v

In [134]:
scaler = StandardScaler()
scaled = scaler.fit_transform(np.array(df_daily.y[:n_train_days]).reshape(-1,1))

In [135]:
predicts_1 = scaler_2.inverse_transform(np.array(res_test[0]))
actuals_1 = scaler_2.inverse_transform(np.array(res_test[1]))
rmse = math.sqrt(mean_squared_error(predicts_1, actuals_1))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 196.082


In [103]:
# load dataset
values = df_total.drop('ds', axis = 1).values
# values = df_combined.drop('ds', axis = 1).values
# integer encode direction
# ensure all data is float
values = values.astype('float32')
# normalize features
# scaler = MinMaxScaler()
scaler = StandardScaler()
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 7, 1)
reframed.head()
# reframed_2 = series_to_supervised(values, 7, 1)
# reframed_2.head()

# drop columns we don't want to predict
y = reframed.iloc[:,-12].values
X = reframed.drop('var1(t)', axis =1).values
# split into train and test sets

n_train_days = 500
n_test_days = 10
train_X = X[:n_train_days,:]
train_y = y[:n_train_days]

val_X= X[n_train_days:-n_test_days,]
val_y= y[n_train_days:-n_test_days]


test_X = X[-n_test_days:,:]
test_y = y[-n_test_days:]



# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

train_and_val_X = np.concatenate((train_X, val_X), axis = 0)

train_and_val_y = np.concatenate((train_y, val_y), axis = 0)

print(train_X.shape, train_y.shape,val_X.shape, val_y.shape ,test_X.shape, test_y.shape)

(500, 1, 31) (500,) (11, 1, 31) (11,) (10, 1, 31) (10,)


In [104]:
LSTM_model_daily_with_sent_1 = Sequential()
LSTM_model_daily_with_sent_1.add(LSTM(64, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), dropout=0.20,recurrent_dropout=0.20))
# LSTM_model_daily_with_sent_1.add(LSTM(128))
LSTM_model_daily_with_sent_1.add(Dense(32))
LSTM_model_daily_with_sent_1.add(Dense(32))
# LSTM_model_daily_with_sent_1.add(LSTM(16, activation='relu'))
LSTM_model_daily_with_sent_1.add(Dense(1))
LSTM_model_daily_with_sent_1.compile(optimizer='adam', loss='mse')

In [105]:
history_daily_with_sent_1 = LSTM_model_daily_with_sent_1.fit(train_X, train_y, 
                    epochs= 100, 
                    batch_size = 32, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

Train on 500 samples, validate on 11 samples
Epoch 1/100
 - 0s - loss: 1.0231 - val_loss: 0.5399
Epoch 2/100
 - 0s - loss: 0.4329 - val_loss: 0.1656
Epoch 3/100
 - 0s - loss: 0.1038 - val_loss: 0.0265
Epoch 4/100
 - 0s - loss: 0.1157 - val_loss: 0.0137
Epoch 5/100
 - 0s - loss: 0.0979 - val_loss: 0.0210
Epoch 6/100
 - 0s - loss: 0.0616 - val_loss: 0.0149
Epoch 7/100
 - 0s - loss: 0.0537 - val_loss: 0.0163
Epoch 8/100
 - 0s - loss: 0.0516 - val_loss: 0.0307
Epoch 9/100
 - 0s - loss: 0.0474 - val_loss: 0.0154
Epoch 10/100
 - 0s - loss: 0.0428 - val_loss: 0.0151
Epoch 11/100
 - 0s - loss: 0.0321 - val_loss: 0.0182
Epoch 12/100
 - 0s - loss: 0.0419 - val_loss: 0.0286
Epoch 13/100
 - 0s - loss: 0.0372 - val_loss: 0.0146
Epoch 14/100
 - 0s - loss: 0.0412 - val_loss: 0.0179
Epoch 15/100
 - 0s - loss: 0.0437 - val_loss: 0.0103
Epoch 16/100
 - 0s - loss: 0.0565 - val_loss: 0.0182
Epoch 17/100
 - 0s - loss: 0.0977 - val_loss: 0.0352
Epoch 18/100
 - 0s - loss: 0.1205 - val_loss: 0.0298
Epoch 19/1

In [106]:
res_sent_1 = make_rolling_window(LSTM_model_daily_with_sent_1, train_X, train_y, val_X, val_y, batch_size = 32, epochs = 2)

Train on 500 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0392 - val_loss: 0.0272
Epoch 2/2
 - 0s - loss: 0.0491 - val_loss: 0.0091
Train on 501 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0701 - val_loss: 0.0092
Epoch 2/2
 - 0s - loss: 0.0568 - val_loss: 0.1398
Train on 502 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0775 - val_loss: 0.0210
Epoch 2/2
 - 0s - loss: 0.0518 - val_loss: 0.0824
Train on 503 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0697 - val_loss: 0.0322
Epoch 2/2
 - 0s - loss: 0.0509 - val_loss: 0.0122
Train on 504 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0684 - val_loss: 0.0141
Epoch 2/2
 - 0s - loss: 0.0432 - val_loss: 0.0361
Train on 505 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0478 - val_loss: 0.0296
Epoch 2/2
 - 0s - loss: 0.0452 - val_loss: 0.0230
Train on 506 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0478 - val_loss: 3.4683e-04
Epoch 2/2
 - 0s - loss: 0.0381 - val_l

In [107]:
res_test_sent_1 = make_rolling_window(LSTM_model_daily_with_sent_1, train_and_val_X, train_and_val_y, test_X, test_y, batch_size = 32, epochs = 2)


Train on 511 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0613 - val_loss: 0.0514
Epoch 2/2
 - 0s - loss: 0.0419 - val_loss: 0.0199
Train on 512 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0581 - val_loss: 0.0124
Epoch 2/2
 - 0s - loss: 0.0412 - val_loss: 0.0410
Train on 513 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0453 - val_loss: 0.0084
Epoch 2/2
 - 0s - loss: 0.0315 - val_loss: 0.0779
Train on 514 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0482 - val_loss: 0.0739
Epoch 2/2
 - 0s - loss: 0.0639 - val_loss: 0.1659
Train on 515 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0897 - val_loss: 0.0098
Epoch 2/2
 - 0s - loss: 0.0630 - val_loss: 0.0917
Train on 516 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0883 - val_loss: 0.0355
Epoch 2/2
 - 0s - loss: 0.0678 - val_loss: 0.0494
Train on 517 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1267 - val_loss: 0.0163
Epoch 2/2
 - 0s - loss: 0.0420 - val_loss:

In [108]:
scaler = StandardScaler()
scaled = scaler.fit_transform(np.array(df_total.y[:n_train_days]).reshape(-1,1))

predicts_sent_1 = scaler.inverse_transform(np.array(res_test_sent_1[0]))
actuals_sent_1 = scaler.inverse_transform(np.array(res_test_sent_1[1]))
rmse = math.sqrt(mean_squared_error(predicts_sent_1, actuals_sent_1))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 824.880


In [109]:
LSTM_model_daily_with_sent_2 = Sequential()
LSTM_model_daily_with_sent_2.add(LSTM(128, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), dropout=0.20,recurrent_dropout=0.20))
# LSTM_model_daily_with_sent_2.add(LSTM(128))
LSTM_model_daily_with_sent_2.add(Dense(64))
LSTM_model_daily_with_sent_2.add(Dense(32))
# LSTM_model_daily_with_sent_2.add(LSTM(16, activation='relu'))
LSTM_model_daily_with_sent_2.add(Dense(1))
LSTM_model_daily_with_sent_2.compile(optimizer='adam', loss='mse')

In [110]:
history_daily_with_sent_2 = LSTM_model_daily_with_sent_2.fit(train_X, train_y, 
                    epochs= 100, 
                    batch_size = 32, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

Train on 500 samples, validate on 11 samples
Epoch 1/100
 - 0s - loss: 0.8770 - val_loss: 0.3737
Epoch 2/100
 - 0s - loss: 0.1116 - val_loss: 0.0078
Epoch 3/100
 - 0s - loss: 0.2579 - val_loss: 0.0418
Epoch 4/100
 - 0s - loss: 0.1695 - val_loss: 0.0124
Epoch 5/100
 - 0s - loss: 0.0762 - val_loss: 0.0115
Epoch 6/100
 - 0s - loss: 0.0514 - val_loss: 0.0052
Epoch 7/100
 - 0s - loss: 0.0415 - val_loss: 0.0036
Epoch 8/100
 - 0s - loss: 0.0432 - val_loss: 0.0038
Epoch 9/100
 - 0s - loss: 0.0439 - val_loss: 0.0171
Epoch 10/100
 - 0s - loss: 0.0442 - val_loss: 0.0049
Epoch 11/100
 - 0s - loss: 0.0601 - val_loss: 0.0379
Epoch 12/100
 - 0s - loss: 0.0670 - val_loss: 0.1174
Epoch 13/100
 - 0s - loss: 0.0901 - val_loss: 0.1535
Epoch 14/100
 - 0s - loss: 0.1088 - val_loss: 0.2114
Epoch 15/100
 - 0s - loss: 0.1513 - val_loss: 0.0537
Epoch 16/100
 - 0s - loss: 0.0642 - val_loss: 0.0065
Epoch 17/100
 - 0s - loss: 0.0676 - val_loss: 0.0074
Epoch 18/100
 - 0s - loss: 0.0539 - val_loss: 0.0038
Epoch 19/1

In [111]:
res_sent_2 = make_rolling_window(LSTM_model_daily_with_sent_2, train_X, train_y, val_X, val_y, batch_size = 32, epochs = 2)


Train on 500 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0483 - val_loss: 0.0462
Epoch 2/2
 - 0s - loss: 0.0468 - val_loss: 0.0096
Train on 501 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0539 - val_loss: 0.0054
Epoch 2/2
 - 0s - loss: 0.0334 - val_loss: 0.0184
Train on 502 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0367 - val_loss: 1.4329e-04
Epoch 2/2
 - 0s - loss: 0.0320 - val_loss: 0.0155
Train on 503 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0282 - val_loss: 0.0026
Epoch 2/2
 - 0s - loss: 0.0283 - val_loss: 0.0045
Train on 504 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0395 - val_loss: 0.0071
Epoch 2/2
 - 0s - loss: 0.0374 - val_loss: 0.0085
Train on 505 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0330 - val_loss: 0.0203
Epoch 2/2
 - 0s - loss: 0.0292 - val_loss: 0.0013
Train on 506 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0302 - val_loss: 0.0091
Epoch 2/2
 - 0s - loss: 0.0301 - val_l

In [112]:
res_test_sent_2 = make_rolling_window(LSTM_model_daily_with_sent_2, train_and_val_X, train_and_val_y, test_X, test_y, batch_size = 32, epochs = 2)


Train on 511 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1028 - val_loss: 0.1071
Epoch 2/2
 - 0s - loss: 0.0624 - val_loss: 0.0788
Train on 512 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1000 - val_loss: 0.0595
Epoch 2/2
 - 0s - loss: 0.0875 - val_loss: 0.0895
Train on 513 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1457 - val_loss: 0.0463
Epoch 2/2
 - 0s - loss: 0.0696 - val_loss: 0.1347
Train on 514 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0843 - val_loss: 0.1446
Epoch 2/2
 - 0s - loss: 0.0869 - val_loss: 0.2691
Train on 515 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1272 - val_loss: 0.0291
Epoch 2/2
 - 0s - loss: 0.0836 - val_loss: 0.0629
Train on 516 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1745 - val_loss: 0.0350
Epoch 2/2
 - 0s - loss: 0.0683 - val_loss: 0.1498
Train on 517 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1038 - val_loss: 0.0100
Epoch 2/2
 - 0s - loss: 0.0641 - val_loss:

In [113]:
predicts_sent_2 = scaler_2.inverse_transform(np.array(res_test_sent_2[0]))
actuals_sent_2 = scaler_2.inverse_transform(np.array(res_test_sent_2[1]))
rmse = math.sqrt(mean_squared_error(predicts_sent_2, actuals_sent_2))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 1036.124


In [114]:
LSTM_model_daily_with_sent_3 = Sequential()
LSTM_model_daily_with_sent_3.add(LSTM(128, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), dropout=0.20,recurrent_dropout=0.20))
# LSTM_model_daily_with_sent_3.add(LSTM(128))
LSTM_model_daily_with_sent_3.add(Dense(64))
LSTM_model_daily_with_sent_3.add(Dense(32))
# LSTM_model_daily_with_sent_3.add(LSTM(16, activation='relu'))
LSTM_model_daily_with_sent_3.add(Dense(1))
LSTM_model_daily_with_sent_3.compile(optimizer='adam', loss='mse')

In [115]:
history_daily_with_sent_3 = LSTM_model_daily_with_sent_3.fit(train_X, train_y, 
                    epochs= 200, 
                    batch_size = 32, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

Train on 500 samples, validate on 11 samples
Epoch 1/200
 - 0s - loss: 0.7846 - val_loss: 0.0423
Epoch 2/200
 - 0s - loss: 0.0613 - val_loss: 0.1057
Epoch 3/200
 - 0s - loss: 0.1252 - val_loss: 0.1258
Epoch 4/200
 - 0s - loss: 0.1687 - val_loss: 0.0472
Epoch 5/200
 - 0s - loss: 0.0530 - val_loss: 0.0134
Epoch 6/200
 - 0s - loss: 0.0473 - val_loss: 0.0092
Epoch 7/200
 - 0s - loss: 0.0426 - val_loss: 0.0142
Epoch 8/200
 - 0s - loss: 0.0423 - val_loss: 0.0122
Epoch 9/200
 - 0s - loss: 0.0400 - val_loss: 0.0388
Epoch 10/200
 - 0s - loss: 0.0545 - val_loss: 0.0451
Epoch 11/200
 - 0s - loss: 0.1251 - val_loss: 0.0716
Epoch 12/200
 - 0s - loss: 0.1378 - val_loss: 0.2116
Epoch 13/200
 - 0s - loss: 0.2736 - val_loss: 0.0294
Epoch 14/200
 - 0s - loss: 0.0432 - val_loss: 0.0047
Epoch 15/200
 - 0s - loss: 0.0382 - val_loss: 0.0189
Epoch 16/200
 - 0s - loss: 0.0473 - val_loss: 0.0097
Epoch 17/200
 - 0s - loss: 0.0327 - val_loss: 0.0162
Epoch 18/200
 - 0s - loss: 0.0363 - val_loss: 0.0290
Epoch 19/2

Epoch 154/200
 - 0s - loss: 0.0407 - val_loss: 0.0273
Epoch 155/200
 - 0s - loss: 0.0544 - val_loss: 0.0375
Epoch 156/200
 - 0s - loss: 0.0436 - val_loss: 0.0240
Epoch 157/200
 - 0s - loss: 0.0649 - val_loss: 0.0336
Epoch 158/200
 - 0s - loss: 0.0478 - val_loss: 0.0474
Epoch 159/200
 - 0s - loss: 0.0694 - val_loss: 0.0282
Epoch 160/200
 - 0s - loss: 0.0347 - val_loss: 0.0148
Epoch 161/200
 - 0s - loss: 0.0434 - val_loss: 0.0410
Epoch 162/200
 - 0s - loss: 0.0368 - val_loss: 0.0198
Epoch 163/200
 - 0s - loss: 0.0596 - val_loss: 0.0292
Epoch 164/200
 - 0s - loss: 0.0488 - val_loss: 0.0334
Epoch 165/200
 - 0s - loss: 0.0731 - val_loss: 0.0219
Epoch 166/200
 - 0s - loss: 0.0498 - val_loss: 0.0169
Epoch 167/200
 - 0s - loss: 0.0716 - val_loss: 0.0309
Epoch 168/200
 - 0s - loss: 0.0397 - val_loss: 0.0101
Epoch 169/200
 - 0s - loss: 0.0579 - val_loss: 0.0222
Epoch 170/200
 - 0s - loss: 0.0395 - val_loss: 0.0244
Epoch 171/200
 - 0s - loss: 0.0441 - val_loss: 0.0352
Epoch 172/200
 - 0s - loss: 

In [116]:
res_sent_3 = make_rolling_window(LSTM_model_daily_with_sent_3, train_X, train_y, val_X, val_y, batch_size = 32, epochs = 2)


Train on 500 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0271 - val_loss: 0.0190
Epoch 2/2
 - 0s - loss: 0.0303 - val_loss: 0.0053
Train on 501 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0460 - val_loss: 0.0028
Epoch 2/2
 - 0s - loss: 0.0428 - val_loss: 0.0369
Train on 502 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0650 - val_loss: 0.0156
Epoch 2/2
 - 0s - loss: 0.0399 - val_loss: 0.0394
Train on 503 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0587 - val_loss: 0.0207
Epoch 2/2
 - 0s - loss: 0.0434 - val_loss: 0.0387
Train on 504 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0675 - val_loss: 0.0237
Epoch 2/2
 - 0s - loss: 0.0515 - val_loss: 0.0600
Train on 505 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0831 - val_loss: 0.0777
Epoch 2/2
 - 0s - loss: 0.0744 - val_loss: 0.0213
Train on 506 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1539 - val_loss: 0.0129
Epoch 2/2
 - 0s - loss: 0.1572 - val_loss:

In [117]:
res_test_sent_3 = make_rolling_window(LSTM_model_daily_with_sent_3, train_and_val_X, train_and_val_y, test_X, test_y, batch_size = 32, epochs = 2)


Train on 511 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0362 - val_loss: 2.4499e-04
Epoch 2/2
 - 0s - loss: 0.0267 - val_loss: 0.0293
Train on 512 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0210 - val_loss: 0.0221
Epoch 2/2
 - 0s - loss: 0.0241 - val_loss: 0.0240
Train on 513 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0229 - val_loss: 0.0177
Epoch 2/2
 - 0s - loss: 0.0214 - val_loss: 5.3915e-04
Train on 514 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0311 - val_loss: 0.0127
Epoch 2/2
 - 0s - loss: 0.0266 - val_loss: 1.1610e-05
Train on 515 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0270 - val_loss: 0.0011
Epoch 2/2
 - 0s - loss: 0.0242 - val_loss: 0.0122
Train on 516 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0305 - val_loss: 5.5052e-04
Epoch 2/2
 - 0s - loss: 0.0290 - val_loss: 0.0029
Train on 517 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0445 - val_loss: 4.9617e-04
Epoch 2/2
 - 0s - loss

In [118]:
predicts_sent_3 = scaler.inverse_transform(np.array(res_test_sent_3[0]))
actuals_sent_3 = scaler.inverse_transform(np.array(res_test_sent_3[1]))
rmse = math.sqrt(mean_squared_error(predicts_sent_3, actuals_sent_3))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 350.940


In [119]:
LSTM_model_daily_with_sent_4 = Sequential()
LSTM_model_daily_with_sent_4.add(LSTM(128, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), dropout=0.20,recurrent_dropout=0.20))
# LSTM_model_daily_with_sent_4.add(LSTM(128))
LSTM_model_daily_with_sent_4.add(Dense(32))
# LSTM_model_daily_with_sent_4.add(LSTM(16, activation='relu'))
LSTM_model_daily_with_sent_4.add(Dense(1))
LSTM_model_daily_with_sent_4.compile(optimizer='adam', loss='mse')

In [120]:
history_daily_with_sent_4 = LSTM_model_daily_with_sent_4.fit(train_X, train_y, 
                    epochs= 200, 
                    batch_size = 32, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

Train on 500 samples, validate on 11 samples
Epoch 1/200
 - 0s - loss: 0.9257 - val_loss: 0.1585
Epoch 2/200
 - 0s - loss: 0.2550 - val_loss: 0.0171
Epoch 3/200
 - 0s - loss: 0.0651 - val_loss: 0.0091
Epoch 4/200
 - 0s - loss: 0.0533 - val_loss: 0.0210
Epoch 5/200
 - 0s - loss: 0.0476 - val_loss: 0.0047
Epoch 6/200
 - 0s - loss: 0.0421 - val_loss: 0.0155
Epoch 7/200
 - 0s - loss: 0.0433 - val_loss: 0.0034
Epoch 8/200
 - 0s - loss: 0.0489 - val_loss: 0.0316
Epoch 9/200
 - 0s - loss: 0.0499 - val_loss: 0.0275
Epoch 10/200
 - 0s - loss: 0.0594 - val_loss: 0.1015
Epoch 11/200
 - 0s - loss: 0.0579 - val_loss: 0.0780
Epoch 12/200
 - 0s - loss: 0.0685 - val_loss: 0.0547
Epoch 13/200
 - 0s - loss: 0.0448 - val_loss: 0.0069
Epoch 14/200
 - 0s - loss: 0.0534 - val_loss: 0.0450
Epoch 15/200
 - 0s - loss: 0.1013 - val_loss: 0.0046
Epoch 16/200
 - 0s - loss: 0.2476 - val_loss: 0.0082
Epoch 17/200
 - 0s - loss: 0.0502 - val_loss: 0.0072
Epoch 18/200
 - 0s - loss: 0.0602 - val_loss: 0.0139
Epoch 19/2

Epoch 154/200
 - 0s - loss: 0.0381 - val_loss: 0.0017
Epoch 155/200
 - 0s - loss: 0.0447 - val_loss: 0.0030
Epoch 156/200
 - 0s - loss: 0.0280 - val_loss: 0.0048
Epoch 157/200
 - 0s - loss: 0.0219 - val_loss: 0.0075
Epoch 158/200
 - 0s - loss: 0.0262 - val_loss: 0.0032
Epoch 159/200
 - 0s - loss: 0.0281 - val_loss: 0.0097
Epoch 160/200
 - 0s - loss: 0.0325 - val_loss: 0.0260
Epoch 161/200
 - 0s - loss: 0.0288 - val_loss: 0.0488
Epoch 162/200
 - 0s - loss: 0.0353 - val_loss: 0.0219
Epoch 163/200
 - 0s - loss: 0.0416 - val_loss: 0.0245
Epoch 164/200
 - 0s - loss: 0.0384 - val_loss: 0.0304
Epoch 165/200
 - 0s - loss: 0.0376 - val_loss: 0.0392
Epoch 166/200
 - 0s - loss: 0.0394 - val_loss: 0.0209
Epoch 167/200
 - 0s - loss: 0.0357 - val_loss: 0.0241
Epoch 168/200
 - 0s - loss: 0.0356 - val_loss: 0.0106
Epoch 169/200
 - 0s - loss: 0.0492 - val_loss: 0.0242
Epoch 170/200
 - 0s - loss: 0.0379 - val_loss: 0.0202
Epoch 171/200
 - 0s - loss: 0.0457 - val_loss: 0.0328
Epoch 172/200
 - 0s - loss: 

In [121]:
res_sent_4 = make_rolling_window(LSTM_model_daily_with_sent_4, train_X, train_y, val_X, val_y, batch_size = 32, epochs = 2)


Train on 500 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0390 - val_loss: 0.0037
Epoch 2/2
 - 0s - loss: 0.0271 - val_loss: 0.0102
Train on 501 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0415 - val_loss: 0.0243
Epoch 2/2
 - 0s - loss: 0.0350 - val_loss: 1.6319e-04
Train on 502 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0519 - val_loss: 0.0382
Epoch 2/2
 - 0s - loss: 0.0305 - val_loss: 2.9034e-04
Train on 503 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0252 - val_loss: 0.0083
Epoch 2/2
 - 0s - loss: 0.0271 - val_loss: 0.0151
Train on 504 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0279 - val_loss: 0.0573
Epoch 2/2
 - 0s - loss: 0.0341 - val_loss: 0.0034
Train on 505 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0339 - val_loss: 0.0233
Epoch 2/2
 - 0s - loss: 0.0264 - val_loss: 0.0124
Train on 506 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0273 - val_loss: 0.0183
Epoch 2/2
 - 0s - loss: 0.0183 - v

In [122]:
res_test_sent_4 = make_rolling_window(LSTM_model_daily_with_sent_4, train_and_val_X, train_and_val_y, test_X, test_y, batch_size = 32, epochs = 2)


Train on 511 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0476 - val_loss: 0.0090
Epoch 2/2
 - 0s - loss: 0.0335 - val_loss: 0.0368
Train on 512 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0395 - val_loss: 0.0221
Epoch 2/2
 - 0s - loss: 0.0324 - val_loss: 0.0546
Train on 513 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0431 - val_loss: 0.0479
Epoch 2/2
 - 0s - loss: 0.0391 - val_loss: 5.7712e-04
Train on 514 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0546 - val_loss: 0.1339
Epoch 2/2
 - 0s - loss: 0.0766 - val_loss: 0.2510
Train on 515 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0664 - val_loss: 0.1150
Epoch 2/2
 - 0s - loss: 0.0606 - val_loss: 0.0710
Train on 516 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0550 - val_loss: 0.0470
Epoch 2/2
 - 0s - loss: 0.0395 - val_loss: 0.0056
Train on 517 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0459 - val_loss: 0.0272
Epoch 2/2
 - 0s - loss: 0.0281 - val_l

In [123]:
predicts_sent_4 = scaler.inverse_transform(np.array(res_test_sent_4[0]))
actuals_sent_4 = scaler.inverse_transform(np.array(res_test_sent_4[1]))
rmse = math.sqrt(mean_squared_error(predicts_sent_4, actuals_sent_4))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 710.364


In [124]:
LSTM_model_daily_with_sent_5 = Sequential()
LSTM_model_daily_with_sent_5.add(LSTM(128, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), dropout=0.25,recurrent_dropout=0.25))
# LSTM_model_daily_with_sent_5.add(LSTM(128))
LSTM_model_daily_with_sent_5.add(Dense(128))
LSTM_model_daily_with_sent_5.add(Dense(128))
# LSTM_model_daily_with_sent_5.add(LSTM(16, activation='relu'))
LSTM_model_daily_with_sent_5.add(Dense(1))
LSTM_model_daily_with_sent_5.compile(optimizer='adam', loss='mse')

In [125]:
history_daily_with_sent_5 = LSTM_model_daily_with_sent_5.fit(train_X, train_y, 
                    epochs= 200, 
                    batch_size = 32, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

Train on 500 samples, validate on 11 samples
Epoch 1/200
 - 0s - loss: 0.9467 - val_loss: 0.1873
Epoch 2/200
 - 0s - loss: 0.0802 - val_loss: 0.1140
Epoch 3/200
 - 0s - loss: 0.0848 - val_loss: 0.1857
Epoch 4/200
 - 0s - loss: 0.1479 - val_loss: 0.2147
Epoch 5/200
 - 0s - loss: 0.3354 - val_loss: 0.6155
Epoch 6/200
 - 0s - loss: 0.6127 - val_loss: 0.1512
Epoch 7/200
 - 0s - loss: 0.2399 - val_loss: 0.1929
Epoch 8/200
 - 0s - loss: 0.1831 - val_loss: 0.1665
Epoch 9/200
 - 0s - loss: 0.2794 - val_loss: 0.0159
Epoch 10/200
 - 0s - loss: 0.0782 - val_loss: 0.0308
Epoch 11/200
 - 0s - loss: 0.0887 - val_loss: 0.0096
Epoch 12/200
 - 0s - loss: 0.0902 - val_loss: 0.0650
Epoch 13/200
 - 0s - loss: 0.0646 - val_loss: 0.0402
Epoch 14/200
 - 0s - loss: 0.0704 - val_loss: 0.0934
Epoch 15/200
 - 0s - loss: 0.0670 - val_loss: 0.0477
Epoch 16/200
 - 0s - loss: 0.0815 - val_loss: 0.0946
Epoch 17/200
 - 0s - loss: 0.0655 - val_loss: 0.1279
Epoch 18/200
 - 0s - loss: 0.0714 - val_loss: 0.0682
Epoch 19/2

Epoch 154/200
 - 0s - loss: 0.0330 - val_loss: 0.0051
Epoch 155/200
 - 0s - loss: 0.0330 - val_loss: 0.0049
Epoch 156/200
 - 0s - loss: 0.0326 - val_loss: 0.0036
Epoch 157/200
 - 0s - loss: 0.0297 - val_loss: 0.0107
Epoch 158/200
 - 0s - loss: 0.0361 - val_loss: 0.0021
Epoch 159/200
 - 0s - loss: 0.0324 - val_loss: 0.0021
Epoch 160/200
 - 0s - loss: 0.0341 - val_loss: 0.0051
Epoch 161/200
 - 0s - loss: 0.0311 - val_loss: 0.0026
Epoch 162/200
 - 0s - loss: 0.0409 - val_loss: 0.0182
Epoch 163/200
 - 0s - loss: 0.0470 - val_loss: 0.0128
Epoch 164/200
 - 0s - loss: 0.0487 - val_loss: 0.0580
Epoch 165/200
 - 0s - loss: 0.0612 - val_loss: 0.0108
Epoch 166/200
 - 0s - loss: 0.0445 - val_loss: 0.0395
Epoch 167/200
 - 0s - loss: 0.0546 - val_loss: 0.0166
Epoch 168/200
 - 0s - loss: 0.0345 - val_loss: 0.0264
Epoch 169/200
 - 0s - loss: 0.0353 - val_loss: 0.0057
Epoch 170/200
 - 0s - loss: 0.0395 - val_loss: 0.0291
Epoch 171/200
 - 0s - loss: 0.0473 - val_loss: 0.0103
Epoch 172/200
 - 0s - loss: 

In [126]:
res_sent_5 = make_rolling_window(LSTM_model_daily_with_sent_5, train_X, train_y, val_X, val_y, batch_size = 32, epochs = 2)


Train on 500 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0991 - val_loss: 0.0133
Epoch 2/2
 - 0s - loss: 0.0528 - val_loss: 0.0437
Train on 501 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0755 - val_loss: 0.0027
Epoch 2/2
 - 0s - loss: 0.0441 - val_loss: 0.0927
Train on 502 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0549 - val_loss: 0.0141
Epoch 2/2
 - 0s - loss: 0.0341 - val_loss: 0.0131
Train on 503 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0274 - val_loss: 4.7805e-09
Epoch 2/2
 - 0s - loss: 0.0241 - val_loss: 0.0139
Train on 504 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0309 - val_loss: 9.9643e-05
Epoch 2/2
 - 0s - loss: 0.0285 - val_loss: 0.0135
Train on 505 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0264 - val_loss: 0.0156
Epoch 2/2
 - 0s - loss: 0.0288 - val_loss: 0.0127
Train on 506 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0390 - val_loss: 1.5492e-05
Epoch 2/2
 - 0s - loss: 0.0441

In [127]:
res_test_sent_5 = make_rolling_window(LSTM_model_daily_with_sent_5, train_and_val_X, train_and_val_y, test_X, test_y, batch_size = 32, epochs = 2)


Train on 511 samples, validate on 1 samples
Epoch 1/2
 - 1s - loss: 0.0440 - val_loss: 0.0918
Epoch 2/2
 - 0s - loss: 0.0366 - val_loss: 0.0053
Train on 512 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0444 - val_loss: 0.0526
Epoch 2/2
 - 0s - loss: 0.0440 - val_loss: 0.0519
Train on 513 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0706 - val_loss: 0.0565
Epoch 2/2
 - 0s - loss: 0.0660 - val_loss: 0.1098
Train on 514 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1153 - val_loss: 0.0905
Epoch 2/2
 - 0s - loss: 0.0642 - val_loss: 0.0899
Train on 515 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1166 - val_loss: 3.2357e-04
Epoch 2/2
 - 0s - loss: 0.0686 - val_loss: 0.2401
Train on 516 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1114 - val_loss: 0.0308
Epoch 2/2
 - 0s - loss: 0.0661 - val_loss: 0.0848
Train on 517 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.1127 - val_loss: 0.0377
Epoch 2/2
 - 0s - loss: 0.0545 - val_l

In [128]:
predicts_sent_5 = scaler.inverse_transform(np.array(res_test_sent_5[0]))
actuals_sent_5 = scaler.inverse_transform(np.array(res_test_sent_5[1]))
rmse = math.sqrt(mean_squared_error(predicts_sent_5, actuals_sent_5))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 977.611
